In [ ]:
# coding=utf-8
import tensorflow as tf
import sys
import numpy as np

In [ ]:
worker_hosts = [
    "worker0.default.svc.cluster.local:2222",
    "worker1.default.svc.cluster.local:2222",
    "localhost:2222",
]

In [ ]:
with tf.device("/job:worker/task:0"):
    with tf.device("/cpu:0"):
        a = tf.constant([1.5, 6.0], name='a')
        b = tf.Variable([1.5, 3.2], name='b')
        c = (a * b) + (a / b)
        d = c * a
        y = tf.assign(b, d)

In [ ]:
with tf.device("/job:worker/task:1"):
    with tf.device("/cpu:0"):
        aa = tf.constant([1.5, 6.0], name='aa')
        bb = tf.Variable([1.5, 3.2], name='bb')
        cc = (aa * bb) + (aa / bb)
        dd = cc + aa
        yy = tf.assign(bb, dd)

In [ ]:
cluster = tf.train.ClusterSpec({"worker": worker_hosts})
server = tf.train.Server(cluster, job_name="worker", task_index=2)

In [ ]:
with tf.Session("grpc://localhost:2222") as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(y))
    print(sess.run(yy))